## Final Project: 

### Objective: 
The 8-puzzle is a sliding puzzle consisting of a 3x3 grid with tiles numbered 1 through 8 and a blank space. The goal is to reach a target configuration by sliding tiles into the blank space. This problem models real-world challenges in robotics and AI.

### Importing Libraries:

In [1]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\multa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import heapq
import copy
import pygame
import sys
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.13.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\multa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


### Constants

In [3]:
WIDTH, HEIGHT = 600, 650
GRID_SIZE = 3
TILE_SIZE = 150
MARGIN = 50
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GRAY = (200, 200, 200)
FONT_SIZE = 40

### Initialize Pygame

In [4]:
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("8-Puzzle Solver")
font = pygame.font.SysFont('Arial', FONT_SIZE)
small_font = pygame.font.SysFont('Arial', 20)

### PuzzleState Class

In [5]:
class PuzzleState:
    """Represents a state of the 8-puzzle board"""
    
    def __init__(self, board, parent=None, move=None, depth=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth
        self.blank_pos = self.find_blank()
        
    def find_blank(self):
        """Find the position of the blank tile (0)"""
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.board[i][j] == 0:
                    return (i, j)
    
    def __eq__(self, other):
        return self.board == other.board
    
    def __hash__(self):
        return hash(str(self.board))
    
    def __lt__(self, other):
        return False
    
    def is_goal(self, goal):
        """Check if current state matches the goal state"""
        return self.board == goal
    
    def get_children(self):
        """Generate all possible next states from current state"""
        children = []
        i, j = self.blank_pos
        moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        
        for di, dj in moves:
            ni, nj = i + di, j + dj
            if 0 <= ni < GRID_SIZE and 0 <= nj < GRID_SIZE:
                new_board = copy.deepcopy(self.board)
                new_board[i][j], new_board[ni][nj] = new_board[ni][nj], new_board[i][j]
                move_desc = f"Move {new_board[i][j]} {'up' if di == -1 else 'down' if di == 1 else 'left' if dj == -1 else 'right'}"
                children.append(PuzzleState(new_board, self, move_desc, self.depth + 1))
        
        return children
    
    def manhattan_distance(self, goal):
        """Calculate Manhattan distance heuristic"""
        distance = 0
        goal_positions = {}
        
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                goal_positions[goal[i][j]] = (i, j)
        
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.board[i][j] != 0:
                    goal_i, goal_j = goal_positions[self.board[i][j]]
                    distance += abs(i - goal_i) + abs(j - goal_j)
        
        return distance
    
    def get_path(self):
        """Reconstruct the path from initial state to current state"""
        path = []
        current = self
        while current.parent is not None:
            path.append(current.move)
            current = current.parent
        return path[::-1]

### Search Algorithm

In [6]:
def a_star_search(initial, goal):
    """A* search algorithm to solve the 8-puzzle"""
    open_set = []
    closed_set = set()
    heapq.heappush(open_set, (0, initial))
    g_score = {initial: 0}
    f_score = {initial: initial.manhattan_distance(goal)}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current.is_goal(goal):
            return current.get_path()
        
        closed_set.add(current)
        
        for child in current.get_children():
            if child in closed_set:
                continue
                
            tentative_g_score = g_score[current] + 1
            
            if child not in g_score or tentative_g_score < g_score[child]:
                child.parent = current
                g_score[child] = tentative_g_score
                f_score[child] = g_score[child] + child.manhattan_distance(goal)
                heapq.heappush(open_set, (f_score[child], child))
    
    return None


### Drawing Functions

In [7]:
def draw_board(board, message=None):
    """Draw the puzzle board on the screen"""
    screen.fill(WHITE)
    
    # Draw title
    title = font.render("8-Puzzle Solver", True, BLACK)
    screen.blit(title, (WIDTH//2 - title.get_width()//2, 10))
    
    # Draw the board
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            rect = pygame.Rect(
                MARGIN + j * TILE_SIZE,
                MARGIN + i * TILE_SIZE + 50,
                TILE_SIZE,
                TILE_SIZE
            )
            pygame.draw.rect(screen, BLUE, rect, 0 if board[i][j] != 0 else 2)
            
            if board[i][j] != 0:
                text = font.render(str(board[i][j]), True, WHITE)
                screen.blit(
                    text,
                    (
                        rect.centerx - text.get_width() // 2,
                        rect.centery - text.get_height() // 2
                    )
                )
    
    # Draw message if any
    if message:
        msg = small_font.render(message, True, BLACK)
        screen.blit(msg, (WIDTH//2 - msg.get_width()//2, HEIGHT - 50))
    
    pygame.display.flip()


### Animation Function

In [8]:
def animate_solution(initial_board, goal_board, path):
    """Animate the solution step by step"""
    current_board = copy.deepcopy(initial_board)
    draw_board(current_board, "Press any key to start solving...")
    waiting = True
    
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN or event.type == pygame.MOUSEBUTTONDOWN:
                waiting = False
    
    solution = a_star_search(PuzzleState(initial_board), goal_board)
    
    if not solution:
        draw_board(current_board, "No solution found!")
        time.sleep(2)
        return
    
    for step in solution:
        # Extract the tile number from the move description
        tile_num = int(''.join(filter(str.isdigit, step)))
        
        # Find the blank position
        blank_i, blank_j = None, None
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if current_board[i][j] == 0:
                    blank_i, blank_j = i, j
                    break
        
        # Find the tile to move
        tile_i, tile_j = None, None
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if current_board[i][j] == tile_num:
                    tile_i, tile_j = i, j
                    break
        
        # Swap the tile with the blank
        current_board[blank_i][blank_j], current_board[tile_i][tile_j] = current_board[tile_i][tile_j], current_board[blank_i][blank_j]
        
        draw_board(current_board, step)
        time.sleep(0.8)
    
    draw_board(current_board, "Puzzle solved!")
    time.sleep(2)


### Main Function

In [ ]:
def main():
    """Main function to run the 8-puzzle solver"""
    # Initial and goal states from the problem
    initial_board = [
        [2, 8, 1],
        [0, 4, 3],
        [7, 6, 5]
    ]
    
    goal_board = [
        [1, 2, 3],
        [8, 0, 4],
        [7, 6, 5]
    ]
    
    running = True
    animate_solution(initial_board, goal_board, None)
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:  # Reset the puzzle
                    animate_solution(initial_board, goal_board, None)
        
        draw_board(initial_board, "Press R to replay solution or close window to exit")
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()